In [1]:
# upgrade pip, make sure you are in the correct directory, and make sure you have the right python
#!python -m pip install --upgrade pip
import os
print(os.getcwd())
os.listdir()
#os.chdir('C:\Users\Christopher\Desktop\DataWarehouseProject\Design\Dimension_Color')
#os.chdir('C:\Users\Christopher\Desktop\Data Warehouse Project\Design\Dimension_Color')
print(os.getcwd())
# Check your version of python to make sure you are running python 3
import platform
print('Pyhton Version: ' + platform.python_version())


d:\django_dashboard_corona\secondDashboard\jcwf_report_credit_analysis
d:\django_dashboard_corona\secondDashboard\jcwf_report_credit_analysis
Pyhton Version: 3.8.10


In [41]:
import psycopg2
import pandas as pd
from pandas import DataFrame
import datetime as dt

# importing data from the data warehouse using psycopg2
conn = psycopg2.connect("host='econometricdatasolutionsdb.postgres.database.azure.com' dbname=postgres user='Christopher@econometricdatasolutionsdb' password='Darkknight17!'")
cur = conn.cursor()

start_date = '2021-08-15' 







cur.execute(""" 

--This is a query that creates a tempeorary table of the Detail line items data, enriched with  product data and color data
--Author: Christopher Gonzalez
--SELECT * FROM jcwf_tran_sales_order_item

DROP TABLE IF EXISTS  jcwf_temp_table;

SELECT  

  sales_order_item_status 
, sales_order_item_product_foreign_key 
, sales_order_item_terms 
, sales_order_item_order_item  
, sales_order_item_po 
, sales_order_item_customer_id_foreign_key 
, sales_order_item_sidemark 
, sales_order_item_entered 
, sales_order_item_credit_ok 
, sales_order_item_printed 
, sales_order_item_labels 
, sales_order_item_packed
, sales_order_item_shipped_date 
, sales_order_item_required 
, sales_order_item_canceled
, sales_order_item_model
, sales_order_item_color_foreign_key 
, sales_order_item_width
, sales_order_item_height 
, sales_order_item_ordered
, sales_order_item_shipped_quantity
, sales_order_item_net_sale  

INTO TABLE jcwf_temp_table

FROM jcwf_fact_sales_order_item
"""

"""
WHERE sales_order_item_entered >= 
"""
+

str("""2021-09-18""")

+
"""
;
SELECT * FROM jcwf_temp_table;
"""
)

rows = cur.fetchall()

print ("\nShow me the databases:\n")
for row in rows:  
    print ("   ", row)

df = DataFrame(rows) # ,columns=[

conn.commit()
conn.close()


UndefinedFunction: operator does not exist: text >= integer
LINE 38: WHERE sales_order_item_entered >= 
                                        ^
HINT:  No operator matches the given name and argument types. You might need to add explicit type casts.


In [ ]:
import psycopg2
import pandas as pd
from pandas import DataFrame
import datetime as dt

# importing data from the data warehouse using psycopg2
conn = psycopg2.connect("host='econometricdatasolutionsdb.postgres.database.azure.com' dbname=postgres user='Christopher@econometricdatasolutionsdb' password='Darkknight17!'")
cur = conn.cursor()

start_date = '2021-08-15' 

cur.execute(""" 

--This is a query that creates a tempeorary table of the Detail line items data, enriched with  product data and color data
--Author: Christopher Gonzalez
--SELECT * FROM jcwf_tran_sales_order_item

DROP TABLE IF EXISTS  jcwf_temp_table;

SELECT  

  sales_order_item_status 
, sales_order_item_product_foreign_key 
, sales_order_item_terms 
, sales_order_item_order_item  
, sales_order_item_po 
, sales_order_item_customer_id_foreign_key 
, sales_order_item_sidemark 
, sales_order_item_entered 
, sales_order_item_credit_ok 
, sales_order_item_printed 
, sales_order_item_labels 
, sales_order_item_packed
, sales_order_item_shipped_date 
, sales_order_item_required 
, sales_order_item_canceled
, sales_order_item_model
, sales_order_item_color_foreign_key 
, sales_order_item_width
, sales_order_item_height 
, sales_order_item_ordered
, sales_order_item_shipped_quantity
, sales_order_item_net_sale  

INTO TABLE jcwf_temp_table

FROM jcwf_fact_sales_order_item
"""

"""
WHERE sales_order_item_entered >= 
"""
+

str("""2021-09-18""")


"""
;
SELECT * FROM jcwf_temp_table;
"""
)

rows = cur.fetchall()

print ("\nShow me the databases:\n")
for row in rows:  
    print ("   ", row)

df = DataFrame(rows) # ,columns=[

conn.commit()
conn.close()


In [92]:
start_date = '\'2021-09-18\''
query = (
str("""

--This is a query that creates a tempeorary table of the Detail line items data, enriched with  product data and color data
--Author: Christopher Gonzalez
--SELECT * FROM jcwf_tran_sales_order_item

DROP TABLE IF EXISTS  jcwf_temp_table;

SELECT  

  sales_order_item_status 
, sales_order_item_product_foreign_key 
, sales_order_item_terms 
, sales_order_item_order_item  
, sales_order_item_po 
, sales_order_item_customer_id_foreign_key 
, sales_order_item_sidemark 
, sales_order_item_entered 
, sales_order_item_credit_ok 
, sales_order_item_printed 
, sales_order_item_labels 
, sales_order_item_packed
, sales_order_item_shipped_date 
, sales_order_item_required 
, sales_order_item_canceled
, sales_order_item_model
, sales_order_item_color_foreign_key 
, sales_order_item_width
, sales_order_item_height 
, sales_order_item_ordered
, sales_order_item_shipped_quantity
, sales_order_item_net_sale  

INTO TABLE jcwf_temp_table

FROM jcwf_fact_sales_order_item
""")
+
str(
"""
WHERE sales_order_item_entered >= 
""")
+
str(start_date)
+
str(
"""
;
SELECT * FROM jcwf_temp_table;
""")
)
print(query)



--This is a query that creates a tempeorary table of the Detail line items data, enriched with  product data and color data
--Author: Christopher Gonzalez
--SELECT * FROM jcwf_tran_sales_order_item

DROP TABLE IF EXISTS  jcwf_temp_table;

SELECT  

  sales_order_item_status 
, sales_order_item_product_foreign_key 
, sales_order_item_terms 
, sales_order_item_order_item  
, sales_order_item_po 
, sales_order_item_customer_id_foreign_key 
, sales_order_item_sidemark 
, sales_order_item_entered 
, sales_order_item_credit_ok 
, sales_order_item_printed 
, sales_order_item_labels 
, sales_order_item_packed
, sales_order_item_shipped_date 
, sales_order_item_required 
, sales_order_item_canceled
, sales_order_item_model
, sales_order_item_color_foreign_key 
, sales_order_item_width
, sales_order_item_height 
, sales_order_item_ordered
, sales_order_item_shipped_quantity
, sales_order_item_net_sale  

INTO TABLE jcwf_temp_table

FROM jcwf_fact_sales_order_item

WHERE sales_order_item_entere

In [93]:
print(query)



--This is a query that creates a tempeorary table of the Detail line items data, enriched with  product data and color data
--Author: Christopher Gonzalez
--SELECT * FROM jcwf_tran_sales_order_item

DROP TABLE IF EXISTS  jcwf_temp_table;

SELECT  

  sales_order_item_status 
, sales_order_item_product_foreign_key 
, sales_order_item_terms 
, sales_order_item_order_item  
, sales_order_item_po 
, sales_order_item_customer_id_foreign_key 
, sales_order_item_sidemark 
, sales_order_item_entered 
, sales_order_item_credit_ok 
, sales_order_item_printed 
, sales_order_item_labels 
, sales_order_item_packed
, sales_order_item_shipped_date 
, sales_order_item_required 
, sales_order_item_canceled
, sales_order_item_model
, sales_order_item_color_foreign_key 
, sales_order_item_width
, sales_order_item_height 
, sales_order_item_ordered
, sales_order_item_shipped_quantity
, sales_order_item_net_sale  

INTO TABLE jcwf_temp_table

FROM jcwf_fact_sales_order_item

WHERE sales_order_item_entere

In [109]:
def start_dater(date):
    start_date = str('\''+str(date)+'\'')
    print(start_date)


start_dater('2021-10-18')




'2021-10-18'




--This is a query that creates a tempeorary table of the Detail line items data, enriched with  product data and color data
--Author: Christopher Gonzalez
--SELECT * FROM jcwf_tran_sales_order_item

DROP TABLE IF EXISTS  jcwf_temp_table;

SELECT

  sales_order_item_status 
, sales_order_item_product_foreign_key 
, product_description
, sales_order_item_terms 
, sales_order_item_order_item  
, sales_order_item_po 
, sales_order_item_customer_id_foreign_key 
, sales_order_item_sidemark 
, sales_order_item_entered 
, sales_order_item_credit_ok 
, sales_order_item_printed 
, sales_order_item_labels 
, sales_order_item_packed
, sales_order_item_shipped_date 
, sales_order_item_required 
, sales_order_item_canceled
, sales_order_item_model
, sales_order_item_color_foreign_key 
, sales_order_item_width
, sales_order_item_height 
, sales_order_item_ordered
, sales_order_item_shipped_quantity
, sales_order_item_net_sale  

INTO TABLE jcwf_temp_table
FROM jcwf_fact_sales_order_item
JOIN jcwf_d

In [3]:
import sys
# importing libraries
import psycopg2
import pandas as pd
from pandas import DataFrame

def simple_query():

    # importing data from the data warehouse using psycopg2
    conn = psycopg2.connect("host='econometricdatasolutionsdb.postgres.database.azure.com' dbname=postgres user='Christopher@econometricdatasolutionsdb' password='Darkknight17!'")
    cur = conn.cursor()
    cur.execute("""
    
    /* Updating transformation table */
    TRUNCATE jcwf_tran_sales_order_item;

    insert INTO jcwf_tran_sales_order_item (

    sales_order_item_product_category  
    , sales_order_item_warehouse  
    , sales_order_item_site 
    , sales_order_item_status 
    , sales_order_item_product 
    , sales_order_item_product_foreign_key
    , sales_order_item_terms 
    , sales_order_item_order_item  
    , sales_order_item_po  
    , sales_order_item_customer_id 
    , sales_order_item_customer_id_foreign_key
    , sales_order_item_customer_name 
    , sales_order_item_sidemark 
    , sales_order_item_entered 
    , sales_order_item_credit_ok  
    , sales_order_item_printed 
    , sales_order_item_labels 
    , sales_order_item_packed 
    , sales_order_item_shipped_date 
    , sales_order_item_required 
    , sales_order_item_canceled 
    , sales_order_item_model 
    , sales_order_item_color_style 
    , sales_order_item_width 
    , sales_order_item_height
    , sales_order_item_ordered 
    , sales_order_item_shipped_quantity 
    , sales_order_item_net_sale
    , sales_order_item_cost_of_good_sold

    )

    SELECT 

    sales_order_item_product_category  
    , sales_order_item_warehouse  
    , sales_order_item_site 
    , sales_order_item_status 
    , sales_order_item_product 
    , (SELECT product_key FROM jcwf_dim_product WHERE jcwf_dim_product.product_id = jcwf_stage_sales_order_item.sales_order_item_product) AS sales_order_item_product_foreign_key
    , sales_order_item_terms 
    , sales_order_item_order_item  
    , sales_order_item_po  
    , sales_order_item_customer_id 
    , (SELECT customer_key FROM jcwf_dim_customer WHERE jcwf_dim_customer.customer_account = jcwf_stage_sales_order_item.sales_order_item_customer_id) AS sales_order_item_customer_id_foreign_key
    , sales_order_item_customer_name 
    , sales_order_item_sidemark 
    , sales_order_item_entered 
    , sales_order_item_credit_ok  
    , sales_order_item_printed 
    , sales_order_item_labels 
    , sales_order_item_packed 
    , sales_order_item_shipped_date 
    , sales_order_item_required 
    , sales_order_item_canceled 
    , sales_order_item_model 
    , sales_order_item_color_style 
    , sales_order_item_width 
    , sales_order_item_height
    , sales_order_item_ordered 
    , sales_order_item_shipped_quantity 
    , sales_order_item_net_sale
    , sales_order_item_cost_of_good_sold

    from jcwf_stage_sales_order_item

    ON CONFLICT ON CONSTRAINT Unique_jcwf_tran_sales_order_item_id DO UPDATE set 

    sales_order_item_product_category = excluded.sales_order_item_product_category
    , sales_order_item_warehouse = excluded.sales_order_item_warehouse
    , sales_order_item_site = excluded.sales_order_item_site
    , sales_order_item_status = excluded.sales_order_item_status
    , sales_order_item_product = excluded.sales_order_item_product
    , sales_order_item_product_foreign_key = excluded.sales_order_item_product_foreign_key
    , sales_order_item_terms = excluded.sales_order_item_terms
    , sales_order_item_order_item = excluded.sales_order_item_order_item
    , sales_order_item_po = excluded.sales_order_item_po
    , sales_order_item_customer_id = excluded.sales_order_item_customer_id
    , sales_order_item_customer_id_foreign_key = excluded.sales_order_item_customer_id_foreign_key
    , sales_order_item_customer_name = excluded.sales_order_item_customer_name
    , sales_order_item_sidemark = excluded.sales_order_item_sidemark
    , sales_order_item_entered = excluded.sales_order_item_entered
    , sales_order_item_credit_ok = excluded.sales_order_item_credit_ok
    , sales_order_item_printed = excluded.sales_order_item_printed
    , sales_order_item_labels = excluded.sales_order_item_labels
    , sales_order_item_packed = excluded.sales_order_item_packed
    , sales_order_item_shipped_date = excluded.sales_order_item_shipped_date
    , sales_order_item_required = excluded.sales_order_item_required
    , sales_order_item_canceled = excluded.sales_order_item_canceled
    , sales_order_item_model = excluded.sales_order_item_model
    , sales_order_item_color_style = excluded.sales_order_item_color_style
    , sales_order_item_width = excluded.sales_order_item_width
    , sales_order_item_height = excluded.sales_order_item_height
    , sales_order_item_ordered = excluded.sales_order_item_ordered
    , sales_order_item_shipped_quantity = excluded.sales_order_item_shipped_quantity
    , sales_order_item_net_sale = excluded.sales_order_item_net_sale
    , sales_order_item_cost_of_good_sold = excluded.sales_order_item_cost_of_good_sold;

    -------------------------------------------------------------------------------------------------------
    /* Updating fact table */
    insert INTO jcwf_fact_sales_order_item (

    sales_order_item_product_category  
    , sales_order_item_warehouse  
    , sales_order_item_site 
    , sales_order_item_status 
    , sales_order_item_product_foreign_key
    , sales_order_item_terms 
    , sales_order_item_order_item  
    , sales_order_item_po  
    , sales_order_item_customer_id_foreign_key 
    , sales_order_item_sidemark 
    , sales_order_item_entered 
    , sales_order_item_credit_ok  
    , sales_order_item_printed 
    , sales_order_item_labels 
    , sales_order_item_packed 
    , sales_order_item_shipped_date 
    , sales_order_item_required 
    , sales_order_item_canceled 
    , sales_order_item_model 
    , sales_order_item_color_foreign_key 
    , sales_order_item_width 
    , sales_order_item_height
    , sales_order_item_ordered 
    , sales_order_item_shipped_quantity 
    , sales_order_item_net_sale
    , sales_order_item_cost_of_good_sold

    )

    SELECT 

    sales_order_item_product_category  
    , sales_order_item_warehouse  
    , sales_order_item_site 
    , sales_order_item_status 
    , sales_order_item_product_foreign_key
    , sales_order_item_terms 
    , sales_order_item_order_item  
    , sales_order_item_po  
    , sales_order_item_customer_id_foreign_key 
    , sales_order_item_sidemark 
    , sales_order_item_entered 
    , sales_order_item_credit_ok  
    , sales_order_item_printed 
    , sales_order_item_labels 
    , sales_order_item_packed 
    , sales_order_item_shipped_date 
    , sales_order_item_required 
    , sales_order_item_canceled 
    , sales_order_item_model 
    , sales_order_item_color_foreign_key 
    , sales_order_item_width 
    , sales_order_item_height
    , sales_order_item_ordered 
    , sales_order_item_shipped_quantity 
    , sales_order_item_net_sale
    , sales_order_item_cost_of_good_sold

    from jcwf_tran_sales_order_item

    ON CONFLICT ON CONSTRAINT Unique_fact_sales_order_item DO UPDATE set 

    sales_order_item_product_category = excluded.sales_order_item_product_category
    , sales_order_item_warehouse = excluded.sales_order_item_warehouse
    , sales_order_item_site = excluded.sales_order_item_site
    , sales_order_item_status = excluded.sales_order_item_status
    , sales_order_item_product_foreign_key = excluded.sales_order_item_product_foreign_key
    , sales_order_item_terms = excluded.sales_order_item_terms
    , sales_order_item_order_item = excluded.sales_order_item_order_item
    , sales_order_item_po = excluded.sales_order_item_po
    , sales_order_item_customer_id_foreign_key = excluded.sales_order_item_customer_id_foreign_key
    , sales_order_item_sidemark = excluded.sales_order_item_sidemark
    , sales_order_item_entered = excluded.sales_order_item_entered
    , sales_order_item_credit_ok = excluded.sales_order_item_credit_ok
    , sales_order_item_printed = excluded.sales_order_item_printed
    , sales_order_item_labels = excluded.sales_order_item_labels
    , sales_order_item_packed = excluded.sales_order_item_packed
    , sales_order_item_shipped_date = excluded.sales_order_item_shipped_date
    , sales_order_item_required = excluded.sales_order_item_required
    , sales_order_item_canceled = excluded.sales_order_item_canceled
    , sales_order_item_model = excluded.sales_order_item_model
    , sales_order_item_color_foreign_key = excluded.sales_order_item_color_foreign_key
    , sales_order_item_width = excluded.sales_order_item_width
    , sales_order_item_height = excluded.sales_order_item_height
    , sales_order_item_ordered = excluded.sales_order_item_ordered
    , sales_order_item_shipped_quantity = excluded.sales_order_item_shipped_quantity
    , sales_order_item_net_sale = excluded.sales_order_item_net_sale
    , sales_order_item_cost_of_good_sold = excluded.sales_order_item_cost_of_good_sold;



    """
    )


    # rows = cur.fetchall()

    # print ("\nShow me the databases:\n")
    # for row in rows:
    #     print ("   ", row)

    # df = DataFrame(rows) # ,columns=[
        
    #   'outsourceditemskey', 'outsourceditemsid', 'Status', 'Product', 'productDescription', 'terms', 'salesOrder',  'lineItem'
    # , 'customerPO', 'CustomerID', 'customername', 'Sidemark'
    # , 'Entered', 'CreditOK', 'Printed', 'Labels' , 'Packed'
    # , 'ShippedDate', 'Required', 'Canceled', 'Model', 'ClrStyle', 'colorDescription', 'WID', 'HGT', 'quantityOrdered', 'quantityShipped'
    # , 'NetSale', 'trackingStatus', 'trackingNumber', 'notes', 'updated'
        
    # ])

    conn.commit()
    conn.close()
    # df
    # return val

if __name__ == '__main__':
    try:
        arg = sys.argv[1]
    except IndexError:
        arg = None

    return_val = simple_query()




In [63]:
if len(customer_key)>1:
    customer_filter = (str("""
    DROP TABLE IF EXISTS  jcwf_temp_table_t;
    SELECT * INTO TABLE jcwf_temp_table_t FROM jcwf_temp_table WHERE sales_order_item_customer_id_foreign_key = """)
    +
    str(customer_key)
    +
    str(""";"""))
else:
    customer_filter =str("""
    DROP TABLE IF EXISTS  jcwf_temp_table_t
    SELECT * INTO TABLE jcwf_temp_table_t FROM jcwf_temp_table;""")
print(customer_filter)


    DROP TABLE IF EXISTS  jcwf_temp_table_t;
    SELECT * INTO TABLE jcwf_temp_table_t FROM jcwf_temp_table WHERE sales_order_item_customer_id_foreign_key = 1953;


In [65]:
import psycopg2
import pandas as pd
from pandas import DataFrame
import datetime as dt

# writing query
customer_key = '1953'
start_date = '\'2021-08-18\''
end_date = '\'2021-10-18\''
if len(customer_key)>1:
    customer_filter = (str("""
    DROP TABLE IF EXISTS  jcwf_temp_table_t;
    SELECT * INTO TABLE jcwf_temp_table_t FROM jcwf_temp_table WHERE sales_order_item_customer_id_foreign_key = """)
    +
    str(customer_key)
    +
    str(""";"""))
else:
    customer_filter =str("""
    DROP TABLE IF EXISTS  jcwf_temp_table_t
    SELECT * INTO TABLE jcwf_temp_table_t FROM jcwf_temp_table;""")

query = (
str("""

--This is a query that creates a tempeorary table of the Detail line items data, enriched with  product data and color data
--Author: Christopher Gonzalez
--SELECT * FROM jcwf_tran_sales_order_item

DROP TABLE IF EXISTS  jcwf_temp_table;

SELECT

  sales_order_item_status 
, sales_order_item_product_foreign_key 
, product_description
, sales_order_item_terms 
, sales_order_item_order_item  
, sales_order_item_po 
, sales_order_item_customer_id_foreign_key 
, sales_order_item_sidemark 
, sales_order_item_entered 
, sales_order_item_credit_ok 
, sales_order_item_printed 
, sales_order_item_labels 
, sales_order_item_packed
, sales_order_item_shipped_date 
, sales_order_item_required 
, sales_order_item_canceled
, sales_order_item_model
, sales_order_item_color_foreign_key 
, sales_order_item_width
, sales_order_item_height 
, sales_order_item_ordered
, sales_order_item_shipped_quantity
, sales_order_item_net_sale  

INTO TABLE jcwf_temp_table
FROM jcwf_fact_sales_order_item
JOIN jcwf_dim_product
ON jcwf_fact_sales_order_item.sales_order_item_product_foreign_key = jcwf_dim_product.product_key
""")
+
str(
"""
WHERE sales_order_item_entered >= 
""")
+
str(start_date)
+
str(
"""
AND sales_order_item_entered <=
"""
+
str(end_date)
# +
# """
# AND sales_order_item_customer_id_foreign_key == ""
# """
+
""";""")
+
# str("""SELECT * FROM jcwf_temp_table;""")
# +
+
str('\"\"\"customer_filter\"\"\"')

# +
# str("""
# SELECT  * FROM jcwf_temp_table_t;
# """)
+
str("""
DROP TABLE IF EXISTS  jcwf_temp_table_2;

SELECT  

  sales_order_item_status 
, sales_order_item_product_foreign_key 
, product_description
, sales_order_item_terms 
, sales_order_item_order_item  
, sales_order_item_po 
, sales_order_item_customer_id_foreign_key 
, Customer_Account
, Customer_Name
, sales_order_item_sidemark 
, sales_order_item_entered 
, sales_order_item_credit_ok 
, sales_order_item_printed 
, sales_order_item_labels 
, sales_order_item_packed
, sales_order_item_shipped_date 
, sales_order_item_required 
, sales_order_item_canceled
, sales_order_item_model
, sales_order_item_color_foreign_key 
, sales_order_item_width
, sales_order_item_height 
, sales_order_item_ordered
, sales_order_item_shipped_quantity
, sales_order_item_net_sale  

INTO TABLE jcwf_temp_table_2
FROM jcwf_temp_table_t
JOIN jcwf_dim_customer
ON jcwf_temp_table_t.sales_order_item_customer_id_foreign_key = jcwf_dim_customer.Customer_key;""")
# +
# str("""SELECT * FROM jcwf_temp_table_2;""")
+
str("""
DROP TABLE IF EXISTS  jcwf_temp_table_3;

SELECT  

  sales_order_item_status 
, sales_order_item_product_foreign_key 
, product_description
, sales_order_item_terms 
, sales_order_item_order_item  
, sales_order_item_po 
, sales_order_item_customer_id_foreign_key 
, Customer_Account
, Customer_Name
, sales_order_item_sidemark 
, sales_order_item_entered 
, sales_order_item_credit_ok 
, sales_order_item_printed 
, sales_order_item_labels 
, sales_order_item_packed
, sales_order_item_shipped_date 
, sales_order_item_required 
, sales_order_item_canceled
, sales_order_item_model
, sales_order_item_color_foreign_key 
, color_name
, color_description
, sales_order_item_width
, sales_order_item_height 
, sales_order_item_ordered
, sales_order_item_shipped_quantity
, sales_order_item_net_sale  

INTO TABLE jcwf_temp_table_3
FROM jcwf_temp_table_2
JOIN jcwf_dim_color
ON jcwf_temp_table_2.sales_order_item_color_foreign_key = jcwf_dim_color.color_key;""")
+
str("""SELECT * FROM jcwf_temp_table_3;""")
)
print(query)


# importing data from the data warehouse using psycopg2
conn = psycopg2.connect("host='econometricdatasolutionsdb.postgres.database.azure.com' dbname=postgres user='Christopher@econometricdatasolutionsdb' password='Darkknight17!'")
cur = conn.cursor()
cur.execute(query) 

rows = cur.fetchall()

print ("\nShow me the databases:\n")
for row in rows:  
    print ("   ", row)

rows = cur.fetchall()

conn.commit()
conn.close()


TypeError: bad operand type for unary +: 'str'